## 1. Parse the xml file
Extract entity and entity context from xml file

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import csv
import string
import pickle

In [2]:
from xml.dom import minidom

NUM_TOKENS = 2
xmldoc = minidom.parse('1.1.test.xml')

In [3]:
def getTokens(node):
    tokens = []
    if node is not None:
        if node.nodeType == node.TEXT_NODE:
            return node.data.split()
        elif node.nodeType == node.ELEMENT_NODE:
            for item in node.childNodes:
                tokens.extend(getTokens(item))
    return tokens

In [4]:
def getWordsBefore(entity_node, num_words):


    words_before = []
    current_node = entity_node

    # Retrieve tokens from previous sibling nodes considering entity collision edge cases
    while True:
        # Calculate remaining offset/number of words per iteration left
        offset = num_words - len(words_before)
        if offset > 0:  
            # Assign previousSibling of the current node as current node per iteration
            current_node = current_node.previousSibling
            if current_node is not None:
                # Get last n tokens from the left
                current_tokens = getTokens(current_node)[-offset:]
            
                current_tokens = [token for token in current_tokens if token not in ['.', ',']]
                # Prepend results
                current_tokens.extend(words_before)
                words_before = current_tokens
                continue  # Proceed iteration
            break  # End iteration if we reach the beginning of abstract content
        break  # End iteration if n words retrieved
    return words_before

In [5]:
def getWordsAfter(entity_node, num_words):

    # return getTokens(entity_node.nextSibling)[-num_words:]

    words_after = []
    current_node = entity_node

    while True:
        # Calculate remaining offset/number of words per iteration left
        offset = num_words - len(words_after)
        if offset > 0:  
            # Assign nextSibling of the current node as current node per iteration
            current_node = current_node.nextSibling
            if current_node is not None:
                # Get first n tokens from the right
                current_tokens = getTokens(current_node)[:offset]
        
                current_tokens = [token for token in current_tokens if token not in ['.', ',']]
                # Append results
                words_after.extend(current_tokens)
                continue  # Proceed iteration
            break  # End iteration if we reach the end of abstract content
        break  # End iteration if n words retrieved
    return words_after

In [95]:
#dictionary to store the context
result = {}
absId = []
entId = []
conxt = []

for entity in xmldoc.getElementsByTagName('entity'):

    textId = entity.parentNode.parentNode.getAttribute("id")
    entityId = entity.getAttribute("id")
    absId.append(textId)
    entId.append(entityId)

    entityWords = getTokens(entity)
    wordsBefore = getWordsBefore(entity, NUM_TOKENS)
    wordsAfter = getWordsAfter(entity, NUM_TOKENS)

    contextTokens = wordsBefore + entityWords + wordsAfter
    contextString = " ".join(contextTokens)
    conxt.append(contextString.lower())

    #constract a dataframe with the abstract Id, entity Id and the context k=2
    test_context= pd.DataFrame({'absID': absId, 'entityID': entId, 'context':conxt})
    
   
    # Build result object (disctionary)

    if not result.get(textId):
        result[textId] = {}

    result[textId][entityId] = contextString

test_context.head(5)
#print(test_context)

absID    entityID                                        context
0  C08-1105  C08-1105.1  in semantic role labeling (srl) arguments are
1  C08-1105  C08-1105.2           labeling (srl) arguments are usually
2  C08-1105  C08-1105.3               in a syntax subtree it arguments
3  C08-1105  C08-1105.4                  to label arguments locally in
4  C08-1105  C08-1105.5                    such a sub-tree rather than

## 2. Preparation of  entiy relations for testing 
1. The above dataframe consists of the abstract Id, entitity Id and the entitiy context with a window of 2, like collecting 2 words before and 2 word after the entity.
2. Read the file with the relation keys given for training. 
3. For each id embedd its entity contexts which was extracted in the previous step. 

In [20]:
test_relations = pd.read_csv("testing_keys.csv") 
test_relations.head()

Enitity_Id1   Entity_Id2  Reverse
0   C08-1105.2   C08-1105.3      NaN
1   C08-1105.4   C08-1105.5      NaN
2  C08-1105.15  C08-1105.16      NaN
3  C08-1105.18  C08-1105.19      NaN
4   L08-1579.1   L08-1579.2  REVERSE

In [66]:
cnt1 = []
for i in range(len(test_relations)):
    for j in range(len(test_context)):
        if test_relations['Enitity_Id1'][i] == test_context['entityID'][j]:
            
            cnt1.append(test_context['context'][j])
    
print(cnt1[1])

to label arguments locally in


In [67]:
len(cnt1)

355

In [69]:
cnt2 = []
for l in range(len(test_relations)):
    for k in range(len(test_context)):
        if test_relations['Entity_Id2'][l] == test_context['entityID'][k]:
            
            cnt2.append(test_context['context'][k])
    
print(cnt2[1])

such a sub-tree rather than


In [70]:
len(cnt2)

355

In [89]:
test_data = pd.DataFrame({'entity_id_1': test_relations['Enitity_Id1'], 'entity_context_1': cnt1,
                     'entity_id_2': test_relations['Entity_Id2'], 'entity_context_2': cnt2})

In [90]:
test_data.head()

entity_id_1                                entity_context_1  entity_id_2  \
0   C08-1105.2            labeling (srl) arguments are usually   C08-1105.3   
1   C08-1105.4                   to label arguments locally in   C08-1105.5   
2  C08-1105.15  approach the anchor group approach achieves an  C08-1105.16   
3  C08-1105.18     that the prediction of mp improves semantic  C08-1105.19   
4   L08-1579.1   of a multilingual translation system based on   L08-1579.2   

                          entity_context_2  
0         in a syntax subtree it arguments  
1              such a sub-tree rather than  
2           achieves an accuracy of 87.75%  
3       mp improves semantic role labeling  
4  on a symbolic parsing technology and on

In [91]:
text_test_data = test_data.drop(columns =['entity_id_1'])
text_test_data = text_test_data.drop(columns =['entity_id_2'])

The entitiy context will be fed to the pretrained model for label prediction

In [92]:
text_test_data.head()

entity_context_1  \
0            labeling (srl) arguments are usually   
1                   to label arguments locally in   
2  approach the anchor group approach achieves an   
3     that the prediction of mp improves semantic   
4   of a multilingual translation system based on   

                          entity_context_2  
0         in a syntax subtree it arguments  
1              such a sub-tree rather than  
2           achieves an accuracy of 87.75%  
3       mp improves semantic role labeling  
4  on a symbolic parsing technology and on

In [94]:
#store the prepared entity context as a pkl file for later use
import pickle
text_test_data.to_pickle("testing_context.pkl")

In [101]:
text_test_data.to_csv('test_data.tsv', sep = '\t', index=False)

In [3]:
#extract the true labels
test_labels = pd.read_csv("true_labels.csv") 
test_labels.head()

Type         id_1         id_2  Reverse
0  PART_WHOLE   C08-1105.2   C08-1105.3      NaN
1  PART_WHOLE   C08-1105.4   C08-1105.5      NaN
2      RESULT  C08-1105.15  C08-1105.16      NaN
3      RESULT  C08-1105.18  C08-1105.19      NaN
4       USAGE   L08-1579.1   L08-1579.2  REVERSE

In [4]:
labels = test_labels['Type']

In [99]:
#labels.head()

In [5]:
test_entity_pair = pd.read_pickle('testing_context.pkl')

In [6]:
test_entity_pair.head()

entity_context_1  \
0            labeling (srl) arguments are usually   
1                   to label arguments locally in   
2  approach the anchor group approach achieves an   
3     that the prediction of mp improves semantic   
4   of a multilingual translation system based on   

                          entity_context_2  
0         in a syntax subtree it arguments  
1              such a sub-tree rather than  
2           achieves an accuracy of 87.75%  
3       mp improves semantic role labeling  
4  on a symbolic parsing technology and on

In [7]:
test_entity_pair['label'] = labels

In [8]:
test_entity_pair.head()

entity_context_1  \
0            labeling (srl) arguments are usually   
1                   to label arguments locally in   
2  approach the anchor group approach achieves an   
3     that the prediction of mp improves semantic   
4   of a multilingual translation system based on   

                          entity_context_2       label  
0         in a syntax subtree it arguments  PART_WHOLE  
1              such a sub-tree rather than  PART_WHOLE  
2           achieves an accuracy of 87.75%      RESULT  
3       mp improves semantic role labeling      RESULT  
4  on a symbolic parsing technology and on       USAGE

In [10]:
test_entity_pair.to_csv('test_data_with_labels.tsv', sep = '\t', index=False)

In [11]:
merge_test = test_entity_pair['entity_context_1'] + ' ' + test_entity_pair['entity_context_2']
testing_pair = pd.DataFrame({'test_pair': merge_test, 'label': test_entity_pair['label'] })
testing_pair.head()

test_pair       label
0  labeling (srl) arguments are usually in a synt...  PART_WHOLE
1  to label arguments locally in such a sub-tree ...  PART_WHOLE
2  approach the anchor group approach achieves an...      RESULT
3  that the prediction of mp improves semantic mp...      RESULT
4  of a multilingual translation system based on ...       USAGE

In [12]:
testing_pair.to_pickle("test_pair.pkl")